In [198]:
import torch
import numpy as np
from torch.nn import functional as F

In [331]:
def np_fast_walsh_hadamard(x, axis, normalize=True):
    """
    Compute Fast Walsh-Hadamard transform in numpy.
    Args:
        x: tensor of shape (a0, a1, ... aN, L, b0, b1, ..., bN).
            L must be a power of two.
        axis: the "L" axis above, aka the axis over which to do the
            Hadamard transform. All other dimensions are left alone;
            data on those dimension do not interact.
        normalize: Whether to normalize the results such that applying
            the transform twice returns to the original input
            value. If True, return values are floats even if input was
            int.
    Returns:
        ret: transformed tensor with same shape as x
    """

    orig_shape = x.shape
    assert axis >= 0 and axis < len(orig_shape), (
        'For a vector of shape %s, axis must be in [0, %d] but it is %d'
        % (orig_shape, len(orig_shape) - 1, axis))
    h_dim = orig_shape[axis]
    h_dim_exp = int(round(np.log(h_dim) / np.log(2)))
    assert h_dim == 2 ** h_dim_exp, (
        'hadamard can only be computed over axis with size that is a power of two, but'
        ' chosen axis %d has size %d' % (axis, h_dim))
    working_shape_pre = [int(np.prod(orig_shape[:axis]))]     # prod of empty array is 1 :)
    working_shape_post = [int(np.prod(orig_shape[axis+1:]))]  # prod of empty array is 1 :)
    working_shape_mid = [2] * h_dim_exp
    working_shape = working_shape_pre + working_shape_mid + working_shape_post
    #print 'working_shape is', working_shape
    ret = x.reshape(working_shape)

    for ii in range(h_dim_exp):
        dim = ii + 1
        arrs = np.split(ret, 2, axis=dim)
        assert len(arrs) == 2
        inter = np.concatenate((arrs[0] + arrs[1], arrs[0] - arrs[1]), axis=dim)
        ret = inter

    if normalize:
        ret = ret / np.sqrt(float(h_dim))

    ret = ret.reshape(orig_shape)

    return ret

def fast_walsh_hadamard_torched(x, axis, normalize=False):
    orig_shape = x.size()
    assert axis >= 0 and axis < len(orig_shape), (
    'For a vector of shape %s, axis must be in [0, %d] but it is %d'
    % (orig_shape, len(orig_shape) - 1, axis))
    h_dim = orig_shape[axis]
    h_dim_exp = int(round(np.log(h_dim) / np.log(2)))
    assert h_dim == 2 ** h_dim_exp, (
    'hadamard can only be computed over axis with size that is a power of two, but'
    ' chosen axis %d has size %d' % (axis, h_dim))
    
    working_shape_pre = [int(np.prod(orig_shape[:axis]))]     # prod of empty array is 1 :)
    working_shape_post = [int(np.prod(orig_shape[axis+1:]))]  # prod of empty array is 1 :)
    working_shape_mid = [2] * h_dim_exp
    working_shape = working_shape_pre + working_shape_mid + working_shape_post
    
    ret = x.view(working_shape)
    
    for ii in range(h_dim_exp):
        dim = ii + 1
        arrs = torch.chunk(ret, 2, dim=dim)
        assert len(arrs) == 2
        ret = torch.cat((arrs[0] + arrs[1], arrs[0] - arrs[1]), axis=dim)
    
    if normalize:
        ret = ret / torch.sqrt(float(h_dim))
    
    ret = ret.view(orig_shape)
    
    return ret

def fastfood_torched(x, DD):

    ll = int(np.ceil(np.log(DD) / np.log(2)))
    LL = 2 ** ll

    BB = torch.FloatTensor(LL).uniform_(0, 2).type(torch.LongTensor)
    BB = (BB * 2 - 1).type(torch.FloatTensor)

    Pi = torch.LongTensor(np.random.permutation(LL))

    GG = torch.FloatTensor(LL,).normal_()

    divisor = torch.sqrt(LL * torch.sum(torch.pow(GG, 2)))

    fastfood_vars = [BB, Pi, GG, divisor]

    dd_pad = F.pad(x, pad=(0, LL - dd), value=0, mode='constant')

    mul_1 = torch.mul(BB, dd_pad)

    mul_2 = fast_walsh_hadamard_torched(mul_1, 0, normalize=False)

    mul_3 = mul_2[Pi]

    mul_4 = torch.mul(mul_3, GG)

    mul_5 = fast_walsh_hadamard_torched(mul_4, 0, normalize=False)

    ret = torch.div(mul_5[:DD], divisor*np.sqrt(float(DD) / LL))

    return ret

## Fastfood below

In [332]:
dd = 4
DD = 20
x = np.array([ 1.6817038, 0.94111705, -0.4930046, 0.9057069 ])

In [333]:
x = torch.from_numpy(x).type(torch.FloatTensor)
print(x.size())
print(x.dtype)

torch.Size([4])
torch.float32


In [335]:
X = fastfood_torched(x, DD)
print(X.size())
print(X)

tensor([ 1.6817,  0.9411, -0.4930,  0.9057])
torch.Size([20])
tensor([ 0.0758, -0.8598,  0.6764,  0.3636, -0.4214, -0.0456, -0.3712,  0.2620,
         0.1477, -0.3099,  0.4032, -0.0119, -0.6375, -0.0612, -0.3593,  0.6843,
        -0.0042,  0.1823,  0.6003, -0.2568])


In [314]:
ll = int(np.ceil(np.log(DD) / np.log(2)))
print(ll)
LL = 2 ** ll
print(LL)

5
32


In [315]:
BB = (torch.FloatTensor(LL).uniform_(0, 2).type(torch.LongTensor) * 2 - 1).type(torch.FloatTensor)
print(BB.size())

torch.Size([32])


In [316]:
Pi = torch.LongTensor(np.random.permutation(LL))
print(Pi)
print(Pi.size())

tensor([31,  2, 17,  7,  8, 16, 29, 21, 15,  3,  9, 12, 14,  0, 28, 22, 13, 30,
         4, 24,  1, 18, 19, 20, 27, 10, 23, 11,  6, 25, 26,  5])
torch.Size([32])


In [317]:
GG = torch.FloatTensor(LL,).normal_()
print(GG)
print(GG.size())

tensor([-0.4825, -0.6431, -0.6793, -1.1537,  0.1646,  0.6058,  0.0722, -1.1346,
        -1.4484, -0.4230, -0.4068,  1.2528,  1.9447, -1.5819,  0.9514,  0.3535,
        -0.2206,  1.6402, -0.8402,  0.2016,  0.1092, -0.6732, -0.1583,  1.0514,
         0.7597,  0.9266,  0.7771,  1.2039, -1.6629,  1.5158, -0.8744, -0.0909])
torch.Size([32])


In [318]:
divisor = torch.sqrt(LL * torch.sum(torch.pow(GG, 2)))
print(divisor)
print(divisor.size())

tensor(30.9100)
torch.Size([])


In [319]:
fastfood_vars = [BB, Pi, GG, divisor]

In [320]:
dd_pad = F.pad(x, pad=(0, LL - dd), value=0, mode='constant')
print(dd_pad)
print(dd_pad.shape)

tensor([ 1.6817,  0.9411, -0.4930,  0.9057,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000])
torch.Size([32])


In [321]:
mul_1 = torch.mul(BB, dd_pad)
print(mul_1)
print(mul_1.size())

tensor([ 1.6817, -0.9411, -0.4930,  0.9057,  0.0000, -0.0000,  0.0000, -0.0000,
         0.0000,  0.0000,  0.0000, -0.0000, -0.0000, -0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000, -0.0000, -0.0000,  0.0000, -0.0000,  0.0000, -0.0000])
torch.Size([32])


In [322]:
mul_2 = fast_walsh_hadamard_torched(mul_1, 0, normalize=False)
print(mul_2)
print(mul_2.shape)
print(mul_2.numpy())

tensor([1.1533, 1.2241, 0.3279, 4.0215, 1.1533, 1.2241, 0.3279, 4.0215, 1.1533,
        1.2241, 0.3279, 4.0215, 1.1533, 1.2241, 0.3279, 4.0215, 1.1533, 1.2241,
        0.3279, 4.0215, 1.1533, 1.2241, 0.3279, 4.0215, 1.1533, 1.2241, 0.3279,
        4.0215, 1.1533, 1.2241, 0.3279, 4.0215])
torch.Size([32])
[1.1532891  1.2241094  0.32788444 4.021532   1.1532891  1.2241094
 0.32788444 4.021532   1.1532891  1.2241094  0.32788444 4.021532
 1.1532891  1.2241094  0.32788444 4.021532   1.1532891  1.2241094
 0.32788444 4.021532   1.1532891  1.2241094  0.32788444 4.021532
 1.1532891  1.2241094  0.32788444 4.021532   1.1532891  1.2241094
 0.32788444 4.021532  ]


In [323]:
mul_3 = mul_2[Pi]

In [324]:
mul_4 = torch.mul(mul_3, GG)

In [325]:
mul_5 = fast_walsh_hadamard_torched(mul_4, 0, normalize=False)

In [326]:
mul_5.size()

torch.Size([32])

In [329]:
ret = torch.div(mul_5[:DD], divisor * np.sqrt(float(DD) / LL ))

In [330]:
ret.size()

torch.Size([20])

In [336]:
ret

tensor([-0.0953, -0.1891, -0.3242, -0.1385, -0.1784, -0.2170, -0.3925, -0.0953,
        -0.5606,  0.1517,  0.8030, -0.2647, -0.4901,  0.2657,  0.6215, -0.3435,
        -1.0986,  0.2238, -0.1148, -0.5346])